In [1]:
!ml Anaconda3
!source activate pannot-dev


In [2]:
from transformers import AutoTokenizer
from pannot.model.language_model.pannot_llama import PannotLlamaForCausalLM
import torch



ModuleNotFoundError: No module named 'transformers'

In [ ]:
from pannot.mm_utils import (
    load_structure_from_pkl,

    tokenizer_protein_token,     # similar to tokenizer_image_token
    get_model_name_from_path      # if reused or overridden in pannot
)

In [ ]:
from pannot.model.builder import load_pretrained_model  


In [ ]:
model_path="/scratch/user/yining_yang/TAMU/PhD/Pannot/checkpoints/pannot-Meta-Llama-3.1-8B-Instruct-finetune-lora-v00"
model_base = "/scratch/user/yining_yang/TAMU/PhD/Pannot/checkpoints/pannot-Meta-Llama-3.1-8B-Instruct-pretrain-v00/checkpoint-24000"
model_name = get_model_name_from_path(model_path)
tokenizer, model, _, context_len = load_pretrained_model(
        model_path, model_base, model_name, use_flash_attn=True
    )

In [ ]:
model.eval()
model_device = next(model.parameters()).device
model_dtype = next(model.parameters()).dtype
if hasattr(model, 'get_seq_tower'):
        model.get_seq_tower().to(model_device)

In [ ]:
eos_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in ["<|end_of_text|>", "<|eom_id|>", "<|eot_id|>"] if tokenizer.convert_tokens_to_ids(t) is not None]
pad_token_id = tokenizer.pad_token_id or eos_token_ids[0]

stopping_keywords = ["<|end_of_text|>", "<|eom_id|>", "<|eot_id|>"]

outputs = []

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig,
    StoppingCriteria
)

from pannot.eval.eval_opi.eval_opi_dataset_pannot import KeywordsStoppingCriteria

stopping_criteria = [KeywordsStoppingCriteria(stopping_keywords, tokenizer, input_ids)]

In [ ]:
temperature = 0.2
top_p = 0.75
num_beams = 1
max_new_tokens = 256



In [1]:
with torch.no_grad():
    output_ids = model.generate(
                    inputs=input_ids,
                    attention_mask=attention_mask,
                    seqs=seqs,
                    seq_attention_mask=seq_attention_masks,
                    do_sample=temperature > 0,
                    temperature=temperature,
                    top_p=top_p,
                    num_beams=num_beams,
                    max_new_tokens=max_new_tokens,
                    use_cache=True,
                    eos_token_id=eos_token_ids,
                    pad_token_id=pad_token_id,
                    stopping_criteria=stopping_criteria
                )


KeyboardInterrupt



The data fprm of OPI test sample:

{"id": "function_test_3", "name": "Protein Function Description", "instruction": "What is the functional description of the protein sequence?", "instances": [{"input": "MSTEGGGRRCQAQVSRRISFSASHRLYSKFLSDEENLKLFGKCNNPNGHGHNYKVVVTVHGEIDPATGMVMNLADLKKYMEEAIMQPLDHKNLDMDVPYFADVVSTTENVAVYIWDNLQKVLPVGVLYKVKVYETDNNIVVYKGE", "output": "Involved in the biosynthesis of tetrahydrobiopterin, an essential cofactor of aromatic amino acid hydroxylases. Catalyzes the transformation of 7,8-dihydroneopterin triphosphate into 6-pyruvoyl tetrahydropterin."}]}


In [ ]:
instruction = "What is the functional description of the protein sequence?"
sequence="MSTEGGGRRCQAQVSRRISFSASHRLYSKFLSDEENLKLFGKCNNPNGHGHNYKVVVTVHGEIDPATGMVMNLADLKKYMEEAIMQPLDHKNLDMDVPYFADVVSTTENVAVYIWDNLQKVLPVGVLYKVKVYETDNNIVVYKGE"

target = "Involved in the biosynthesis of tetrahydrobiopterin, an essential cofactor of aromatic amino acid hydroxylases. Catalyzes the transformation of 7,8-dihydroneopterin triphosphate into 6-pyruvoyl tetrahydropterin."

In [ ]:
PROT_TOKEN_INDEX = -300
DEFAULT_PROT_TOKEN = "<prot>"
DEFAULT_PROT_PATCH_TOKEN = "<prot_patch>"
DEFAULT_PROT_START_TOKEN = "<prot_start>"
DEFAULT_PROT_END_TOKEN = "<prot_end>"
PROT_PLACEHOLDER = "<prot-placeholder>"

SEQ_TOKEN_INDEX = -330
DEFAULT_SEQ_TOKEN = "<seq>"
DEFAULT_SEQ_PATCH_TOKEN = "<seq_patch>"
DEFAULT_SEQ_START_TOKEN = "<seq_start>"
DEFAULT_SEQ_END_TOKEN = "<seq_end>"

SEQUENCE_PLACEHOLDER = '<seq-placeholder>'

seq_token_se = DEFAULT_SEQ_START_TOKEN + DEFAULT_SEQ_TOKEN + DEFAULT_SEQ_END_TOKEN

if SEQUENCE_PLACEHOLDER in instruction:
                prompt = instruction.replace(SEQUENCE_PLACEHOLDER, seq_token_se)
else:
                prompt = seq_token_se + "\n" + instruction


In [ ]:
from pannot.conversation import conv_templates


if "llama" in model_name.lower():
        conv_mode = "pannot_llama_2"
elif "mistral" in model_name.lower():
        conv_mode = "mistral_instruct"
elif "v1.6-34b" in model_name.lower():
        conv_mode = "chatml_direct"
elif "v1" in model_name.lower():
        conv_mode = "pannot_v1"
else:
        conv_mode = "pannot_v0"

conv = conv_templates[conv_mode].copy()
conv.append_message(conv.roles[0], prompt)
conv.append_message(conv.roles[1], None)
full_prompt = conv.get_prompt()


In [ ]:
tokenized_prompt = tokenizer_protein_token(full_prompt, tokenizer, return_tensors="pt").unsqueeze(0)
input_ids = tokenized_prompt.to(model.device)
attention_mask = torch.ones_like(input_ids)


In [ ]:

if hasattr(model, 'get_seq_tower'):
        tokenized_seq = model.get_seq_tower().tokenize(sequence)
        seq_input_id = tokenized_seq["input_ids"].squeeze(0).to(model.device)
        seq_attention_mask = tokenized_seq["attention_mask"].squeeze(0).to(model.device)
        seqs = [seq_input_id]
        seq_attention_masks = [seq_attention_mask]
else:
        seqs = None
        seq_attention_masks = None

In [ ]:
from pannot.eval.eval_opi.eval_opi_dataset_pannot import KeywordsStoppingCriteria


stopping_criteria = [KeywordsStoppingCriteria(stopping_keywords, tokenizer, input_ids)]

In [ ]:
temperature = 0.5
top_p = 0.75
num_beams = 1
max_new_tokens = 256
with torch.no_grad():
    output_ids = model.generate(
                    inputs=input_ids,
                    attention_mask=attention_mask,
                    seqs=seqs,
                    seq_attention_mask=seq_attention_masks,
                    do_sample=temperature > 0,
                    temperature=temperature,
                    top_p=top_p,
                    num_beams=num_beams,
                    max_new_tokens=max_new_tokens,
                    use_cache=True,
                    eos_token_id=eos_token_ids,
                    pad_token_id=pad_token_id,
                    stopping_criteria=stopping_criteria
                )
pred = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

In [ ]:
# # Load base LLM
# model = PannotLlamaForCausalLM.from_pretrained(
#     # "/scratch/user/yining_yang/TAMU/PhD/Pannot/local_pretrained_llm/Meta-Llama-3.1-8B-Instruct",
#     "/scratch/user/yining_yang/TAMU/PhD/Pannot/checkpoints/pannot-Meta-Llama-3.1-8B-Instruct-pretrain-v00",
#     torch_dtype=torch.bfloat16,
#     device_map="auto",  # automatic single-device or CPU+GPU placement
# )




# # Load adapter weights
# adapter_weights = torch.load("TAMU/PhD/Pannot/checkpoints/pannot-Meta-Llama-3.1-8B-Instruct-pretrain-v00/mm_projector.bin", map_location="cpu")
# model.load_state_dict(adapter_weights, strict=False)

# # Tokenizer
# tokenizer = AutoTokenizer.from_pretrained("/scratch/user/yining_yang/TAMU/PhD/Pannot/local_pretrained_llm/Meta-Llama-3.1-8B-Instruct")

# Input prompt
prompt = "Describe the function of this protein."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

print("Has sequence encoder:", hasattr(model, "seq_tower"))
print("Has structure encoder:", hasattr(model, "struc_tower"))

# Optional: check inner modules
print("Seq tower type:", type(model.seq_tower))
print("Struc tower type:", type(model.struc_tower))


# Run generation
outputs = model.generate(
    input_ids=input_ids,
    max_new_tokens=256,
    do_sample=False,
    temperature=0.7
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))